In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_1_ = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20s_2020_onwards.csv')
df_2_ = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20is_2020_onwards.csv')

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_2_[(df_2_.batting_team=='India')&(df_2_.bowling_team=='England')].start_date.max(), df_1_.start_date.max(),\
df_2_.start_date.max()

('2025-02-02', '2025-06-03', '2025-06-01')

In [4]:
df = pd.concat([df_1_,df_2_],axis=0)

df = df.drop_duplicates().reset_index(drop=True)

cols_replace_zero = ['wides','noballs','byes','legbyes','penalty']
cols_replace_empty = ['other_wicket_type','other_player_dismissed']

for column in cols_replace_zero:
    df[column] = df[column].replace(np.nan, 0)

for column in cols_replace_empty:
    df[column] = df[column].replace(np.nan, '')

for col in df.columns:
    if df[col].nunique()<=1:
        df.drop(col, axis=1, inplace=True)

In [5]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year'],
      dtype='object')

### checking which column has nan

In [6]:
df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

df['wicket_type'].replace(np.nan, '', inplace=True)
df['player_dismissed'].replace(np.nan, '', inplace=True)

cols_to_string = ['batting_team','bowling_team','striker','non_striker','bowler','wicket_type',
                  'player_dismissed']

for col in cols_to_string:
    df[col] = df[col].astype(str)

## creating some necessary columns

In [7]:
df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

df['legality'] = (df.wides+df.noballs)==0

df['bowler_wicket'] = (df.wicket_type=='caught') | (df.wicket_type=='bowled') | (df.wicket_type=='lbw') |\
                        (df.wicket_type=='stumped')|(df.wicket_type=='caught and bowled') | \
                        (df.wicket_type=='hit wicket') 

df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

df['islegal']= df['legality'].apply(lambda x: 1 if x==True else 0)
df['isDotforBatter']= df['runs_off_bat'].apply(lambda x: 1 if x==0 else 0)
df['isOne']= df['runs_off_bat'].apply(lambda x: 1 if x==1 else 0)
df['isTwo']= df['runs_off_bat'].apply(lambda x: 1 if x==2 else 0)
df['isThree']= df['runs_off_bat'].apply(lambda x: 1 if x==3 else 0)

df['isStrikeRotation'] = df['runs_off_bat'].apply(lambda x: 1 if ((x==1)|(x==2)|(x==3)| (x==5)|(x==7)) else 0)
#assuming all 4s and 6s are boundaries hit
df['isFour']= df['runs_off_bat'].apply(lambda x: 1 if x==4 else 0)
df['isSix']= df['runs_off_bat'].apply(lambda x: 1 if x==6 else 0)
df['isBowlerWicket']= df['bowler_wicket'].apply(lambda x: 1 if x==True else 0)




    

In [8]:
df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

df['is_striker_Out'] = (df.striker==df.player_dismissed)

df['is_striker_Out']= df['is_striker_Out'].apply(lambda x: 1 if x==True else 0)

df['is_nonstriker_Out'] = (df.non_striker==df.player_dismissed)

df['is_nonstriker_Out']= df['is_nonstriker_Out'].apply(lambda x: 1 if x==True else 0)

df['isWicket'] = df['wicket_type'].apply(lambda x:1 if len(x)>0 else 0)

df['total_runs'] = df.runs_off_bat+df.extras

## next part of work :: building the crucial cols

### creation of runs_scored, wickets_down, legal_balls_bowled

In [9]:
# Sort the DataFrame by 'match_id' and 'innings' to ensure proper ordering
df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

# Calculate the cumulative sum for 'total_runs' within each group of 'match_id' and 'innings'
df['runs_scored'] = df.groupby(['match_id', 'innings'])['total_runs'].cumsum()

# Reset the cumulative sum for each new 'match_id' or 'innings'
df.loc[df['innings'].diff().eq(1), 'runs_scored'] = df['total_runs']

# Reset the cumulative sum for the first row of each 'match_id'
df.loc[df['match_id'].ne(df['match_id'].shift(1)), 'runs_scored'] = df['total_runs']

# Optional: If you want to fill NaN values in 'runs_scored' with 0
df['runs_scored'] = df['runs_scored'].fillna(0)

# Now, 'runs_scored' contains the cumulative sum of 'total_runs' for each innings in a match_id


In [10]:
df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

In [11]:
# Calculate the cumulative sum for 'isWicket' within each group of 'match_id' and 'innings'
df['wickets_down'] = df.groupby(['match_id', 'innings'])['isWicket'].cumsum()

# Reset the cumulative sum for each new 'match_id' or 'innings'
df.loc[df['innings'].diff().eq(1), 'wickets_down'] = df['isWicket']

# Reset the cumulative sum for the first row of each 'match_id'
df.loc[df['match_id'].ne(df['match_id'].shift(1)), 'wickets_down'] = df['isWicket']

# Optional: If you want to fill NaN values in 'wickets_down' with 0
df['wickets_down'] = df['wickets_down'].fillna(0)

# Now, 'wickets_down' contains the cumulative sum of 'isWicket' for each innings in a match_id


In [12]:
df = df.sort_values(by=['start_date', 'match_id', 'innings', 'ball']).reset_index(drop=True)

In [13]:
# Calculate cumulative sum for 'islegal' within each group of 'match_id' and 'innings'
df['legal_balls_bowled'] = df.groupby(['match_id', 'innings'])['islegal'].cumsum()

# Reset cumulative sum when there's a change in 'match_id' or 'innings'
# By shifting down, we check if either 'match_id' or 'innings' changed in the previous row
df.loc[df['match_id'] != df['match_id'].shift(1), 'legal_balls_bowled'] = df['islegal']
df.loc[df['innings'] != df['innings'].shift(1), 'legal_balls_bowled'] = df['islegal']

# Optional: Fill NaN values with 0, if there are any NaNs after resetting
df['legal_balls_bowled'] = df['legal_balls_bowled'].fillna(0)

In [14]:
# Calculate the cumulative sum for 'islegal' within each group of 'match_id' and 'innings'
df['legal_balls_bowled'] = df.groupby(['start_date','match_id', 'innings'])['islegal'].cumsum()

# Reset the cumulative sum for each new 'match_id' or 'innings'
df.loc[df['innings'].diff().eq(1), 'legal_balls_bowled'] = df['islegal']

# Reset the cumulative sum for the first row of each 'match_id'
df.loc[df['match_id'].ne(df['match_id'].shift(1)), 'legal_balls_bowled'] = df['islegal']

# Optional: If you want to fill NaN values in 'legal_balls_bowled' with 0
df['legal_balls_bowled'] = df['legal_balls_bowled'].fillna(0)

# Now, 'legal_balls_bowled' contains the cumulative sum of 'islegal' for each innings in a match_id


In [15]:
df['lbb_fl'] = 6*(df['ball']+1)>df['legal_balls_bowled']

df.shape[0]-df['lbb_fl'].sum()

0

In [16]:
df = df.reset_index(drop=True)

### filtering on the matches with completed 1st innings only

In [17]:
#Find match_ids that meet the condition in innings=1
filtered_match_ids = df[(df['innings'] == 1) & (df['legal_balls_bowled'].max() < 120) & (df['wickets_down'].max() < 10)]['match_id'].unique()

#Filter the DataFrame to exclude rows with those match_ids
df = df[~df['match_id'].isin(filtered_match_ids)]

### creation of run_rate, target

In [18]:
df['run_rate'] = df.apply(lambda row: (6 * row['runs_scored'] / row['legal_balls_bowled']) \
                          if row['legal_balls_bowled'] > 0 else (6 + row['runs_scored']), axis=1)


In [19]:
# Create a DataFrame with the maximum runs scored in innings=1 for each match_id
max_runs_innings_1 = df[df['innings'] == 1].groupby('match_id')['runs_scored'].max().reset_index()

# Merge this information into innings=2 rows of the original DataFrame
df = df.merge(max_runs_innings_1, on='match_id', suffixes=('', '_max_innings_1'), how='left')

# Create the 'target' column based on the maximum runs scored in innings=1
df['target'] = df.apply(lambda row: row['runs_scored_max_innings_1'] + 1 if row['innings'] == 2 else np.nan, axis=1)

# Drop the intermediate 'runs_scored_max_innings_1' column if needed
df.drop('runs_scored_max_innings_1', axis=1, inplace=True)

df.sample(4)

### filtering on the matches with not-shortened 2nd innings only

In [20]:
lost = 0
shortened = 0
won = 0
shortened_matches = []

for match in df.match_id.unique():
    max_achieved_2nd_innings = df[(df.innings==2) & (df.match_id==match)]['runs_scored'].max()
    target_2nd_innings = df[(df.innings==2) & (df.match_id==match)]['target'].max()
    wickets_2nd_innings = df[(df.innings==2) & (df.match_id==match)]['wickets_down'].max()
    balls_2nd_innings = df[(df.innings==2) & (df.match_id==match)]['legal_balls_bowled'].max()
    
    if max_achieved_2nd_innings<target_2nd_innings:
        if (wickets_2nd_innings<10) & (balls_2nd_innings<120):
            shortened = shortened+1
            shortened_matches.append(match)
            
        else :
            lost = lost+1
               
    else:
        won = won+1

In [21]:
len(shortened_matches)

225

In [22]:
# Find match_ids that meet the condition in innings=2
filtered_match_ids = shortened_matches

#target not achieved, not all out, some balls remain


# Filter the DataFrame to exclude rows with those match_ids
df = df[~df['match_id'].isin(filtered_match_ids)]

In [23]:
df.match_id.nunique()

4573

In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
df = df[~(df.innings>2)]

### creation of legal_balls_remaining, runs_remaining, reqd_run_rate

In [26]:
df['legal_balls_remaining'] = 120 - df['legal_balls_bowled']
df['runs_remaining'] = df.apply(lambda row: (row['target'] - row['runs_scored']) if row['innings'] == 2 \
                                else np.nan, axis=1)


df.loc[df.runs_remaining<0, 'runs_remaining'] = 0

df['reqd_run_rate'] = df.apply(lambda row: (6* row['runs_remaining'] / row['legal_balls_remaining']) \
                            if row['legal_balls_remaining'] > 0 \
                                else 6+row['runs_remaining'], axis=1)
df.loc[df.innings==1, 'reqd_run_rate'] = np.nan

### creation of last_fow, partnership_runs

In [27]:
df.sample(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,year,legality,bowler_wicket,islegal,isDotforBatter,isOne,isTwo,isThree,isStrikeRotation,isFour,isSix,isBowlerWicket,is_striker_Out,is_nonstriker_Out,isWicket,total_runs,runs_scored,wickets_down,legal_balls_bowled,lbb_fl,run_rate,target,legal_balls_remaining,runs_remaining,reqd_run_rate
463832,1334896,2022/23,2022-10-19,"Senwes Park, Potchefstroom",1,5.6,Dolphins,North West,JT Smuts,B Parsons,D Potgieter,2,0,0.0,0.0,0.0,0.0,0.0,,,,,2022,True,False,1,0,0,1,0,1,0,0,0,0,0,0,2,49,1,35,True,8.4,NaN,85,NaN,NaN


In [28]:
import pandas as pd
import numpy as np

# Initialize the 'last_fow_run_rate' column with NaN
df['last_fow_run_rate'] = np.nan

# Initialize variables to keep track of the last wickets down, innings, and match_id
last_wickets_down = None
current_innings = None
current_match_id = None
last_fow_run_rate = None
##
last_fow = None

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    wickets_down = row['wickets_down']
    run_rate = row['run_rate']
    runs_scored = row['runs_scored']

    # Check if innings or match_id has changed
    if innings != current_innings or match_id != current_match_id:
        last_fow_run_rate = None  # Reset last_fow_run_rate to None
        ##
        last_fow = None

    # Update the current 'wickets_down', 'innings', 'match_id'
    current_wickets_down = wickets_down
    current_innings = innings
    current_match_id = match_id

    # Update the 'last_fow_run_rate' column if 'wickets_down' changes or becomes 1
    if wickets_down != last_wickets_down and wickets_down != 0:
        last_fow_run_rate = run_rate
        ##
        last_fow = runs_scored

    # Update 'last_fow_run_rate' immediately when 'wickets_down' becomes 1 or changes
    if wickets_down == 1 or (last_wickets_down != 0 and wickets_down != last_wickets_down):
        df.at[index, 'last_fow_run_rate'] = run_rate
        ##
        df.at[index, 'last_fow'] = runs_scored
        last_fow_run_rate = run_rate
        ##
        last_fow = runs_scored

    # Update 'last_wickets_down'
    last_wickets_down = wickets_down

# Now, 'last_fow_run_rate' should update immediately when 'wickets_down' becomes 1 or changes, and retain its value in other cases, resetting every match and innings as NaN


In [130]:
%%time

partnership_runs = []
for match in df.match_id.unique():
    for inning in df[df.match_id==match].innings.unique():
        partnership_runs.append(df[df.match_id==match][df.innings==inning]['runs_scored'].iloc[0])
        for row in range(1, len(df[df.match_id==match][df.innings==inning])):
            previous_fow = df[df.match_id==match][df.innings==inning]['last_fow'].iloc[row-1]
            to_append = df[df.match_id==match][df.innings==inning]['runs_scored'].iloc[row] - \
                                        (previous_fow if pd.notna(previous_fow) else 0)
            partnership_runs.append(to_append)

<timed exec>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CPU times: user 2h 50min 16s, sys: 15min 1s, total: 3h 5min 17s
Wall time: 1h 16min 41s


In [34]:
partnership_runs = []

for (match, inning), group in df.groupby(['match_id', 'innings']):
    group = group.sort_values('runs_scored').reset_index(drop=True)  # ensure proper order
    
    for i in range(len(group)):
        if i == 0:
            partnership_runs.append(group['runs_scored'].iloc[i])
        else:
            prev_fow = group['last_fow'].iloc[i - 1]
            prev_fow_val = prev_fow if pd.notna(prev_fow) else 0
            runs = group['runs_scored'].iloc[i] - prev_fow_val
            partnership_runs.append(runs)


In [35]:
df['partnership_runs'] = partnership_runs

In [42]:
df.shape

(1056601, 50)

### creation of striker_runs, non_striker_runs

In [41]:
df.sample(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,year,legality,bowler_wicket,islegal,isDotforBatter,isOne,isTwo,isThree,isStrikeRotation,isFour,isSix,isBowlerWicket,is_striker_Out,is_nonstriker_Out,isWicket,total_runs,runs_scored,wickets_down,legal_balls_bowled,lbb_fl,run_rate,target,legal_balls_remaining,runs_remaining,reqd_run_rate,last_fow_run_rate,last_fow,partnership_runs
682074,1383480,2023/24,2023-10-19,"JSCA International Stadium Complex, Ranchi",1,17.4,Gujarat,Goa,RV Patel,Saurav Chauhan,SP Tari,4,0,0.0,0.0,0.0,0.0,0.0,,,,,2023,True,False,1,0,0,0,0,0,1,0,0,0,0,0,4,180,5,106,True,10.188679,NaN,14,NaN,NaN,NaN,NaN,83.0


In [43]:
df['lbb_fl'] = 6*(df['ball']+1)>df['legal_balls_bowled']

df.shape[0]-df['lbb_fl'].sum()

0

In [44]:
df[df.lbb_fl==0].head(10)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,year,legality,bowler_wicket,islegal,isDotforBatter,isOne,isTwo,isThree,isStrikeRotation,isFour,isSix,isBowlerWicket,is_striker_Out,is_nonstriker_Out,isWicket,total_runs,runs_scored,wickets_down,legal_balls_bowled,lbb_fl,run_rate,target,legal_balls_remaining,runs_remaining,reqd_run_rate,last_fow_run_rate,last_fow,partnership_runs


In [45]:
%%time

striker_runs = []
non_striker_runs = []
i = 0


for match in df.match_id.unique():
    i = i+1
    df_part = df[df.match_id==match]
    for inning in df_part.innings.unique():
        first_loc = df_part[df_part.innings==inning].index[0]

        for row in (df_part[df_part.innings==inning].index):
            s_r = df_part[df_part.innings==inning][df_part.striker==df_part.striker.loc[row]].loc[first_loc:row]\
                                                    .runs_off_bat.sum()
            n_s_r = df_part[df_part.innings==inning][df_part.striker==df_part.non_striker.loc[row]].loc[first_loc:row]\
                                                    .runs_off_bat.sum()
            striker_runs.append(s_r)
            non_striker_runs.append(n_s_r)

<timed exec>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CPU times: user 19min 7s, sys: 4.03 s, total: 19min 11s
Wall time: 19min 5s


In [46]:
df['striker_runs'] = striker_runs
df['non_striker_runs'] = non_striker_runs

In [47]:
df['isno'] = (df.noballs>0)

df['is_faced_by_batter'] = (df.islegal==1) | (df.isno==True)


df['is_faced_by_batter'].sum()

1019113

In [48]:
df.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/intermediate.csv',
             index=None)

df = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/intermediate.csv')

In [49]:
%%time

striker_balls = []
non_striker_balls = []
i = 0


for match in df.match_id.unique():
    i = i+1
    df_part = df[df.match_id==match]
    for inning in df_part.innings.unique():
        first_loc = df_part[df_part.innings==inning].index[0]

        for row in (df_part[df_part.innings==inning].index):
            s_b = df_part[df_part.innings==inning][df_part.striker==df_part.striker.loc[row]].loc[first_loc:row]\
                                                    .is_faced_by_batter.sum()
            n_s_b = df_part[df_part.innings==inning][df_part.striker==df_part.non_striker.loc[row]].loc[first_loc:row]\
                                                    .is_faced_by_batter.sum()
            striker_balls.append(s_b)
            non_striker_balls.append(n_s_b)

i

df['striker_balls'] = striker_balls
df['non_striker_balls'] = non_striker_balls

#df['striker_SR'] = 

<timed exec>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CPU times: user 19min 11s, sys: 4.09 s, total: 19min 15s
Wall time: 19min 10s


### creation of last_fow_run_rate, 

In [50]:
df.shape

(1056601, 56)

In [51]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year', 'legality', 'bowler_wicket',
       'islegal', 'isDotforBatter', 'isOne', 'isTwo', 'isThree',
       'isStrikeRotation', 'isFour', 'isSix', 'isBowlerWicket',
       'is_striker_Out', 'is_nonstriker_Out', 'isWicket', 'total_runs',
       'runs_scored', 'wickets_down', 'legal_balls_bowled', 'lbb_fl',
       'run_rate', 'target', 'legal_balls_remaining', 'runs_remaining',
       'reqd_run_rate', 'last_fow_run_rate', 'last_fow', 'partnership_runs',
       'striker_runs', 'non_striker_runs', 'isno', 'is_faced_by_batter',
       'striker_balls', 'non_striker_balls'],
      dtype='object')

In [52]:
# Initialize the 'last_fow_run_rate' column with NaN
df['last_fow_run_rate'] = np.nan

# Initialize variables to keep track of the last innings and match_id
current_innings = None
current_match_id = None

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    is_wicket = row['isWicket']
    run_rate = row['run_rate']

    # Check if innings or match_id has changed
    if innings != current_innings or match_id != current_match_id:
        df.at[index, 'last_fow_run_rate'] = np.nan  # Reset 'last_fow_run_rate' to NaN
        current_innings = innings  # Update current_innings
        current_match_id = match_id  # Update current_match_id

    # Update 'last_fow_run_rate' whenever a wicket falls (isWicket == 1)
    if is_wicket == 1:
        df.at[index, 'last_fow_run_rate'] = run_rate

# Now, 'last_fow_run_rate' should start with NaN, update whenever a wicket falls, and reset to NaN every innings and match


In [53]:
# Sort the DataFrame by 'match_id', 'innings' for forward filling
df.sort_values(by=['match_id', 'start_date', 'innings', 'ball'], inplace=True)

# Group by 'match_id' and 'innings' and forward-fill NaN values within each group
df['last_fow_run_rate'] = df.groupby(['match_id', 'innings'])['last_fow_run_rate'].fillna(method='ffill')

# Reset the DataFrame index
df.reset_index(drop=True, inplace=True)


#### correcting the last_fow in some cases

In [54]:

# Sort the DataFrame by 'match_id', 'innings' for forward filling
df.sort_values(by=['match_id', 'start_date', 'innings', 'ball'], inplace=True)

# Group by 'match_id' and 'innings' and forward-fill NaN values within each group
df['last_fow'] = df.groupby(['match_id', 'innings'])['last_fow'].fillna(method='ffill')

# Reset the DataFrame index
df.reset_index(drop=True, inplace=True)


In [55]:
df.shape

(1056601, 56)

In [56]:
df = df.sort_values(by = ['match_id', 'start_date', 'innings', 'ball'])

df.tail(600)

In [57]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year', 'legality', 'bowler_wicket',
       'islegal', 'isDotforBatter', 'isOne', 'isTwo', 'isThree',
       'isStrikeRotation', 'isFour', 'isSix', 'isBowlerWicket',
       'is_striker_Out', 'is_nonstriker_Out', 'isWicket', 'total_runs',
       'runs_scored', 'wickets_down', 'legal_balls_bowled', 'lbb_fl',
       'run_rate', 'target', 'legal_balls_remaining', 'runs_remaining',
       'reqd_run_rate', 'last_fow_run_rate', 'last_fow', 'partnership_runs',
       'striker_runs', 'non_striker_runs', 'isno', 'is_faced_by_batter',
       'striker_balls', 'non_striker_balls'],
      dtype='object')

## next part of work :: seggregation and filtering 

In [58]:
df.sample(n=1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,year,legality,bowler_wicket,islegal,isDotforBatter,isOne,isTwo,isThree,isStrikeRotation,isFour,isSix,isBowlerWicket,is_striker_Out,is_nonstriker_Out,isWicket,total_runs,runs_scored,wickets_down,legal_balls_bowled,lbb_fl,run_rate,target,legal_balls_remaining,runs_remaining,reqd_run_rate,last_fow_run_rate,last_fow,partnership_runs,striker_runs,non_striker_runs,isno,is_faced_by_batter,striker_balls,non_striker_balls
616977,1369568,2023,2023-08-27,"Warner Park, Basseterre, St Kitts",2,1.3,Jamaica Tallawahs,Guyana Amazon Warriors,BA King,KSA McKenzie,R Shepherd,0,0,0.0,0.0,0.0,0.0,0.0,caught,BA King,,,2023,True,True,1,1,0,0,0,0,0,0,1,1,0,1,0,1,1,9,True,0.666667,211.0,111,210.0,11.351351,0.666667,1.0,47.0,1,0,False,True,6,3


### innings division

In [59]:
df_1 = df[df.innings==1]
df_2 = df[df.innings==2]

In [60]:
df_1.shape, df_2.shape

((558081, 56), (498520, 56))

### working with both innings

#### creation of fours_hit, sixes_hit

In [61]:
# Sort the DataFrame by 'match_id', 'innings' for forward filling
df_1.sort_values(by=['match_id', 'start_date', 'innings', 'ball'], inplace=True)

# Group by 'match_id' and 'innings' and forward-fill NaN values within each group
df_1['fours'] = df_1.groupby(['match_id', 'innings'])['isFour'].cumsum()
df_1['sixes'] = df_1.groupby(['match_id', 'innings'])['isSix'].cumsum()

# Reset the DataFrame index
df_1.reset_index(drop=True, inplace=True)


/Users/roumyadas/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_15444/2831839755.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['fours'] = df_1.groupby(['match_id', 'innings'])['isFour'].cumsum()
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_15444/2831839755.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

df_1.head(300)

#### creation of fours_hit, sixes_hit

In [62]:
# Sort the DataFrame by 'match_id', 'innings' for forward filling
df_2.sort_values(by=['match_id', 'start_date', 'innings', 'ball'], inplace=True)

# Group by 'match_id' and 'innings' and forward-fill NaN values within each group
df_2['fours'] = df_2.groupby(['match_id', 'innings'])['isFour'].cumsum()
df_2['sixes'] = df_2.groupby(['match_id', 'innings'])['isSix'].cumsum()

# Reset the DataFrame index
df_2.reset_index(drop=True, inplace=True)


/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_15444/2393252262.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['fours'] = df_2.groupby(['match_id', 'innings'])['isFour'].cumsum()
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_15444/2393252262.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['sixes'] = df_2.groupby(['match_id', 'innings'])['isSix'].cumsum()


In [63]:

# Create a dictionary to store the maximum runs_scored for each match_id and innings
max_runs_dict = {}

# Iterate through the DataFrame to populate the max_runs_dict
for index, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    runs_scored = row['runs_scored']
    
    if match_id not in max_runs_dict:
        max_runs_dict[match_id] = {'innings_1_max': 0, 'innings_2_max': 0}
    
    if innings == 1 and runs_scored > max_runs_dict[match_id]['innings_1_max']:
        max_runs_dict[match_id]['innings_1_max'] = runs_scored
    elif innings == 2 and runs_scored > max_runs_dict[match_id]['innings_2_max']:
        max_runs_dict[match_id]['innings_2_max'] = runs_scored

# Create a function to determine the 'Win_FL' value based on max_runs_dict
def calculate_win_fl(row):
    match_id = row['match_id']
    innings = row['innings']
    
    if innings == 1:
        if max_runs_dict[match_id]['innings_1_max'] > max_runs_dict[match_id]['innings_2_max']:
            return 1
        elif max_runs_dict[match_id]['innings_1_max'] < max_runs_dict[match_id]['innings_2_max']:
            return


In [64]:
%%time

# Initialize a dictionary to store player-to-number mappings for each match_id
player_to_number = {}

# Initialize a list to store player_number
player_number = [1]

# Function to update player-to-number mappings
def update_player_mapping(player):
    player_to_number[player] = player_number[0]
    player_number[0] += 1

# Lists to store batting positions for striker and non-striker
striker_positions = []
non_striker_positions = []

# Initialize current match_id
current_match_id = None

# Iterate through DataFrame rows
for index, row in df_1.iterrows():
    #print(index)
    
    match_id = row['match_id']
    striker = row['striker']
    non_striker = row['non_striker']
    
    # Check if it's a new match_id
    if current_match_id != match_id:
        current_match_id = match_id
        player_to_number.clear()
        player_number[0] = 1
    
    # Check if striker has been encountered before in this match
    if striker not in player_to_number:
        update_player_mapping(striker)
    
    # Check if non-striker has been encountered before in this match
    if non_striker not in player_to_number:
        update_player_mapping(non_striker)
    
    # Append player numbers to the respective lists
    striker_positions.append(player_to_number[striker])
    non_striker_positions.append(player_to_number[non_striker])

# Create new columns 'striker_batting_position' and 'non_striker_batting_position'
df_1['striker_batting_position'] = striker_positions
df_1['non_striker_batting_position'] = non_striker_positions

# Now, 'df_1_mod' contains the desired columns with player numbering based on their first appearance in each match


CPU times: user 13.6 s, sys: 661 ms, total: 14.2 s
Wall time: 14.5 s


<timed exec>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
%%time

# Initialize a dictionary to store player-to-number mappings for each match_id
player_to_number = {}

# Initialize a list to store player_number
player_number = [1]

# Function to update player-to-number mappings
def update_player_mapping(player):
    player_to_number[player] = player_number[0]
    player_number[0] += 1

# Lists to store batting positions for striker and non-striker
striker_positions = []
non_striker_positions = []

# Initialize current match_id
current_match_id = None

# Iterate through DataFrame rows
for index, row in df_2.iterrows():
   
    
    match_id = row['match_id']
    striker = row['striker']
    non_striker = row['non_striker']
    
    # Check if it's a new match_id
    if current_match_id != match_id:
        current_match_id = match_id
        player_to_number.clear()
        player_number[0] = 1
    
    # Check if striker has been encountered before in this match
    if striker not in player_to_number:
        update_player_mapping(striker)
    
    # Check if non-striker has been encountered before in this match
    if non_striker not in player_to_number:
        update_player_mapping(non_striker)
    
    # Append player numbers to the respective lists
    striker_positions.append(player_to_number[striker])
    non_striker_positions.append(player_to_number[non_striker])

# Create new columns 'striker_batting_position' and 'non_striker_batting_position'
df_2['striker_batting_position'] = striker_positions
df_2['non_striker_batting_position'] = non_striker_positions

# Now, 'df_2_mod' contains the desired columns with player numbering based on their first appearance in each match


CPU times: user 12.1 s, sys: 479 ms, total: 12.5 s
Wall time: 12.7 s


<timed exec>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [66]:
df.shape

(1056601, 56)

In [67]:
total_runs_df = df.pivot_table(index=['match_id'], columns='innings', values='runs_scored',\
                               aggfunc='max').reset_index(drop=False)

total_runs_df.columns = ['match_id', 'inn_1_total', 'inn_2_total']

In [68]:
total_runs_df.shape

(4573, 3)

In [69]:
total_runs_df['inn_1_win_fl'] = 0

total_runs_df.loc[total_runs_df.inn_1_total>total_runs_df.inn_2_total, 'inn_1_win_fl'] = 1
total_runs_df.loc[total_runs_df.inn_1_total==total_runs_df.inn_2_total, 'inn_1_win_fl'] = 0.5



In [70]:
total_runs_df['inn_2_win_fl'] = 1 - total_runs_df['inn_1_win_fl']

df_1.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/inn_1_i.csv',
             index=None)

df_2.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/inn_2_i.csv',
             index=None)

In [71]:
df_1.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/inn_1.csv',
             index=None)

df_2.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/inn_2.csv',
             index=None)

df[(df.batting_team=='India')&(df.bowling_team=='South Africa')].tail()

In [72]:
df[df.striker=='Tilak Varma'][df.batting_team=='India'].start_date.value_counts()

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_15444/1081569298.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.striker=='Tilak Varma'][df.batting_team=='India'].start_date.value_counts()


2024-11-13    59
2025-01-25    57
2024-11-15    52
2023-08-06    45
2023-08-08    38
2023-11-28    28
2023-10-06    27
2024-01-11    23
2024-11-10    22
2023-08-03    22
2023-08-13    19
2024-11-08    18
2025-01-22    16
2025-01-28    15
2025-02-02    15
2023-11-23    10
2023-10-03    10
2023-08-12     6
2023-11-26     3
2023-08-20     2
2023-12-14     1
2025-01-31     1
Name: start_date, dtype: int64

In [73]:
df.start_date.max(), df_2.start_date.max()

('2025-06-03', '2025-06-03')